# Pacotes

In [1]:
%pip install openpyxl

   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   ---------------------------------------- 250.0/250.0 kB 5.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import glob
import os
 
import openpyxl

# 1) Coleta dos dados

### 1.1) Leitura dos arquivos

#### OBS: Escolhi utilizar os arquivos csv pela facilidade de leitura e poder utilizar chunks de dados

In [4]:
def get_csv_files(years: range = range(2000, 2021), directory: str = "../data", chunksize: int = 10000) -> list: 
    dataframes_list = []

    for year in years:
        file_name = f"GERACAO_USINA_{year}.csv"
        file_path = os.path.join(directory, file_name)

        print(f"\nLendo o arquivo {file_name}...")

        if glob.glob(file_path):
            # Leia o arquivo XLSX
            df = pd.read_csv(file_path, chunksize=chunksize, delimiter=";", encoding="utf-8", low_memory=False)

            # Adicione o DataFrame à lista
            dataframes_list.append(df)
        else:
            raise Exception(f"O arquivo {file_name} não foi encontrado.")
            

    return dataframes_list

In [56]:
try: 
    dataframes = get_csv_files(range(2000, 2001), "../data", 10000)
except Exception as e:
    print(f"Erro: {e}")


Lendo o arquivo GERACAO_USINA_2000.csv...


### 1.2) Lendo Dataframes por chunk

In [61]:
def reading_chunks(df, cols: list = ["din_instante", "id_subsistema", "val_geracao", "nom_tipousina"]):
    for chunk in df:
        print(chunk.shape)
        print(chunk.info())
        display(chunk.head(5))
        

In [62]:
cols = ["din_instante", "id_subsistema", "val_geracao", "nom_tipousina"]

reading_chunks(dataframes[0], cols)

(10000, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 30000 to 39999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   din_instante            10000 non-null  object 
 1   id_subsistema           10000 non-null  object 
 2   nom_subsistema          10000 non-null  object 
 3   id_estado               10000 non-null  object 
 4   nom_estado              10000 non-null  object 
 5   cod_modalidadeoperacao  10000 non-null  object 
 6   nom_tipousina           10000 non-null  object 
 7   nom_tipocombustivel     10000 non-null  object 
 8   nom_usina               10000 non-null  object 
 9   ceg                     10000 non-null  object 
 10  val_geracao             10000 non-null  float64
dtypes: float64(1), object(10)
memory usage: 859.5+ KB
None


,din_instante,id_subsistema,nom_subsistema,id_estado,nom_estado,cod_modalidadeoperacao,nom_tipousina,nom_tipocombustivel,nom_usina,ceg,val_geracao
30000,2000-01-12 06:00:00,S,SUL,RS,RIO GRANDE DO SUL,TIPO I,TÉRMICA,Carvão,Charqueadas,UTE.CM.RS.000760-9.01,20.8
30001,2000-01-12 06:00:00,S,SUL,SC,SANTA CATARINA,Pequenas Usinas (Tipo III),HIDROELÉTRICA,Hidráulica,Pequenas Centrais Hidroelétricas da Celesc,-,50.0
30002,2000-01-12 06:00:00,S,SUL,SC,SANTA CATARINA,TIPO I,TÉRMICA,Carvão,J. Lacerda C,UTE.CM.SC.027094-6.01,353.4
30003,2000-01-12 06:00:00,S,SUL,SC,SANTA CATARINA,TIPO I,TÉRMICA,Carvão,J. Lacerda A,UTE.CM.SC.001260-2.01,109.2
30004,2000-01-12 06:00:00,S,SUL,SC,SANTA CATARINA,TIPO I,TÉRMICA,Carvão,J. Lacerda B,UTE.CM.SC.027093-8.01,252.0


# 2) Preparação inicial dos dados

### 2.1) Seleção dos dados

#### OBS: Agrupei os dados para a região sudeste e com foco na produção de energia por meio de hidrelétricas

# 3) Analise estatística

# 4) Pré-processamento dos dados

# 5) Modelo de predição

### 5.1) Divisão dos dados

### 5.2) Arquitetando rede LTMS

# 6) Avaliação do modelo